In [6]:
from experiments.experiment_type import ExperimentType, get_experiments_in_group, get_config,get_common_experiment_group
from experiments.experiment_config import ExperimentConfig
from experiments.experiment import Experiment


def run_experiment_group(experiment_group):
    group:list[ExperimentType] = get_experiments_in_group(experiment_group)
    conf:ExperimentConfig = get_config(group[0])
    conf.evaluation_interval = conf.epochs
    conf.train_model = True
    conf.use_tensorboard = True
    conf.use_tensorboard = True
    conf.detailed_logging = False

    for exp in group:
        conf.experiment_name = exp.name
        Exp = Experiment(experiment_type= exp, config=conf)
        Exp.run()
        Exp.save_results(overwrite=True)



In [10]:

import os
def verify_p(ex_type):
    ps = ex_type.get_data_paths()
    test = ps["test"]
    train = ps["train"]
    print(os.path.exists(test))
    print(os.path.exists(train))

In [13]:
g = get_experiments_in_group("3_3")
len(g)

30